<a href="https://colab.research.google.com/github/SattikBhaumik/Analyzing_Exoplanets/blob/main/Exoplanet_Main_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightcurve Finder




In [ ]:
!pip install lightkurve

In [ ]:
import lightkurve as lk
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
#uer-input Star Name
N_star = input('Enter Kepler Star name:')

#downloading all the observations till date
data = lk.search_lightcurve(N_star, cadence='long')
lc_all = data.download_all()             

In [ ]:
#printing the whole dataset for observation quarters, years etc.
data  

In [ ]:
#plotting the whole data      
lc_all.plot() 

In [ ]:
#plotting the flux versus time data after stitching and flattening all datasets 
lc = lc_all.stitch().flatten(window_length=1001).remove_outliers()
lc.plot()

In [ ]:
lc.flux

In [ ]:
#note down the F_trans value for later use
F_trans = np.amin(lc.flux)
F_trans

In [ ]:
#creating array for searching period of exoplanet
period = np.linspace(1, 20, 10000)  

In [ ]:
#plotting the BoxLeastSqaures Periodogram
bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500)
bls.plot()

In [ ]:
#finding period from bls periodogram and transit depth and duratin for masking
planet_period = bls.period_at_max_power 
planet_t = bls.transit_time_at_max_power 
planet_d = bls.duration_at_max_power

#print planet orbital period, note it down for later use
planet_period

In [ ]:
#checking if the period is correct by using to get a folded light-curve
ax = lc.fold(period=planet_period, epoch_time=planet_t).scatter()
ax.set_xlim(-5, 5)

In [ ]:
# Create a cadence mask using the BLS parameters
planet_mask = bls.get_transit_mask(period=planet_period, transit_time=planet_t, duration=planet_d)

#creating masked light-curve for additional transit curves
masked_lc = lc[~planet_mask]
ax = masked_lc.scatter()
lc[planet_mask].scatter(ax=ax, c='r', label='Masked')

In [ ]:
# Create a BLS model using the BLS parameters
planet_model = bls.get_transit_model(period=planet_period, transit_time=planet_t,  duration=planet_d)

#using the model we plot the folded light curve
ax = lc.fold(planet_period, planet_t).scatter()
planet_model.fold(planet_period, planet_t).plot(ax=ax, c='r', lw=2)
ax.set_xlim(-5, 5)

In [ ]:
# we will now look for another planet using BoxLeastSqaures method
# we will look for further peaks in the periodogram plot indicative of a second planet
period = np.linspace(1, 300, 10000)
bls_new = masked_lc.to_periodogram('bls', period=period, frequency_factor=500)
bls_new.plot()

In [ ]:
#If we get a distinctive peak, then we have another planet in that system 
planet_new_period = bls_new.period_at_max_power
planet_new_t = bls_new.transit_time_at_max_power
planet_new_d = bls_new.duration_at_max_power

# print the orbital period value
planet_new_period

In [ ]:
#we plot the folded light-curve for this new planet
ax_new = masked_lc.fold(planet_new_period, planet_new_t).scatter()
masked_lc.fold(planet_new_period, planet_new_t).bin(.1).plot(ax=ax_new, c='b', lw=2, label='Binned Flux')  #binned using data points for 0.1 days
ax_new.set_xlim(-5, 5)

In [ ]:
# Create a BLS model using the BLS parameters for the new planet
planet_new_model = bls.get_transit_model(period=planet_new_period, transit_time=planet_new_t, duration=planet_new_d)


#using the model and the previous one, we plot both into a single plot 
ax_new = lc.scatter()
planet_model.plot(ax=ax_new, c='b', label='Planet 1 Transit Data')
planet_new_model.plot(ax=ax_new, c='r', label='Planet 2 Transit Data')

In [ ]:
F = np.amin(planet_new_model.flux)
F

# Finding Planet Parameters

In [ ]:
#constants
G=6.6743e-11
au=1.496e+11
M_earth=5.97216787e+24
M_jup=1.8981246e+27
M_sun=1.98840987e+30
R_earth=6378100
R_jup=71492000
R_sun=695700000
import math
pi= math.pi
al= 0.07
F_tot= 1

#Earth Data
E_rho= 5.51       #density in g cm^-3
E_ve= 11.19       #escape velocity in km s^-1
E_t= 288.00       #temp in K

#weight exponents
w_R= 0.57         #radius parameter
w_Rho= 1.07       #density parameter
w_VE= 0.70        #escape velocity parameter
w_T= 5.58         #surface temperature parameter


In [ ]:
#taking user input of Star data from database
def STAR():                                        
  global M_star
  M_star=input('Enter Star Mass in terms of Sun:')
  M_star = float(M_star)*M_sun
  global R_star
  R_star=input('Enter Star Radius in Sun:')
  R_star = float(R_star)*R_sun
  global T_star
  T_star=input('Enter Star Temperature in K:')
  T_star = float(T_star)

STAR()

In [ ]:
#taking user input of Planet data after TOPCAT analysis
def PLANET():
  global P
  P = input('Enter Planet Orbital Period in days:')
  P = float(P)*86400 
  global F_Trans
  F_Trans = input('Enter Planet Flux:')
  F_Trans = float(F_Trans)
  global Mass
  Mass= input('Enter Planet Mass in terms of Earth:')
  Mass= float(Mass)*M_earth

PLANET()

In [ ]:
#calculating planet components
def Exoplanet():
  global a,a1
  a=math.pow((G*M_star*P*P/(4*pi*pi)), (1/3))
  a1=a/au

  global R,R1,R2
  R=float(R_star*math.sqrt(1-(F_Trans/F_tot)))
  R1=float(R/R_earth)
  R2=float(R/R_jup)

  global T,TC
  T=float(T_star*math.pow(((1-al)*R_star**2/a**2), (1/4)))
  T=T/math.sqrt(2)
  TC=float(T-273.15)

  global M1,M2
  M1=float(Mass/M_jup)            #in terms of jup
  M2=float(Mass/M_earth)          #in terms of earth

  global Vol, Dens, Dens1      #check density conversion
  Vol=float((4/3)*pi*R*R*R)
  Dens1=float(Mass/Vol)            #kg m3    
  Dens=Dens1/1000               #g cm3

  global Vems, VeKms
  Vems= float(math.sqrt(2*G*Mass/R))
  VeKms= Vems/1000
  
Exoplanet()

In [ ]:
#calculating ESI parameters
ESI_R = math.pow((1-math.fabs((R-R_earth)/(R+R_earth))), w_R)              #radius parameter
ESI_Rho = math.pow((1-math.fabs((Dens-E_rho)/(Dens+E_rho))), w_Rho)        #density parameter
ESI_VE = math.pow((1-math.fabs((VeKms-E_ve)/(VeKms+E_ve))), w_VE)          #escape velocity parameter
ESI_T = math.pow((1-math.fabs((T-E_t)/(T+E_t))), w_T)                      #temperature parameter

#calculating final ESI's
ESI_Int = math.sqrt(ESI_R*ESI_Rho)         #interior ESI
ESI_Sur = math.sqrt(ESI_VE*ESI_T)          #surface ESI
ESI_Global = math.sqrt(ESI_Int*ESI_Sur)    #final ESI determining habitability

In [ ]:
#printing 
#planet data
print('Planet Orbital Radius in m:',a)
print('Planet Orbital Radius in AU:', a1)

print('Planet Radius in m:', R)
print('Planet Radius in terms of Earth:', R1)
print('Planet Radius in terms of Jupiter:', R2)

print('Planet Temperature in kelvin:', T)
print('Planet Temperature in celsius:', TC)

print('Planet Mass in kg:', Mass)
print('Planet Mass in terms of Jupiter:', M1)
print('Planet Mass in terms of Earth:', M2)

print('Planet Density in kg m^-3:', Dens1)
print('Planet Density in g cm^-3:', Dens) #check 

print('Escape Velocity of Planet in m/s:', Vems)
print('Escape Velocity of Planet in km/s:', VeKms)

#ESI data
print('Interior ESI:', ESI_Int)
print('Surface ESI:', ESI_Sur)
print('Global ESI:', ESI_Global)